In [21]:
import pandas as pd
import numpy as np
#from parameter_estimator import ParameterEstimator as pe
from robot import RobotDescription
import utils
from collections import deque
import cv2
import random
import math as m
from itertools import combinations
import pickle
from pytransform3d.transform_manager import *
import datetime
from scipy.spatial.transform import Rotation as R

In [5]:
# functions to convert from and to spherical coordinates
def asSpherical(x, y, z):
    r = m.sqrt(x*x + y*y + z*z)
    theta = m.acos(z/r)*180 / m.pi  # to degrees
    phi = m.atan2(y, x)*180 / m.pi
    return r, theta, phi


def asCartesian(r, theta, phi):
    theta = theta * m.pi/180  # to radian
    phi = phi * m.pi/180
    x = r * m.sin(theta) * m.cos(phi)
    y = r * m.sin(theta) * m.sin(phi)
    z = r * m.cos(theta)
    return x, y, z

In [50]:
# generate the poses of markers
ids = list(range(30))  # marker ids to use
dict_T_WM = {}

for marker_id in ids:  # generate random poses for virtual markers
    r = np.random.normal(3, 0.12, 1)[0]
    theta = np.random.uniform(0, 90, 1)[0]
    phi   = np.random.uniform(0, 360, 1)[0]
    x, y, z = asCartesian(r, theta, phi)
    print(f"{x:.3f} {y:.3f} {z:.3f}")
    dict_T_WM[marker_id]= utils.H_rvec_tvec(np.array([0.0, 0, 0]), np.array([x, y, z]))


1.268 -2.288 1.122
2.413 -0.824 1.534
1.224 1.512 2.130
0.204 1.637 2.352
1.673 1.421 2.387
-0.314 0.315 2.995
3.155 -0.722 0.663
0.458 -2.503 1.637
-0.077 0.031 3.182
2.194 0.349 2.111
0.887 -0.613 2.643
-0.102 0.050 3.088
1.209 1.579 2.068
0.359 2.022 2.260
-0.311 -0.695 2.956
-2.714 0.471 0.722
0.378 2.771 0.777
-0.700 -0.555 2.560
1.471 -2.350 1.363
-1.417 -0.971 2.466
2.752 0.126 0.021
-0.661 1.493 2.262
0.022 -0.103 2.889
-0.349 0.034 2.933
-0.389 -0.225 2.699
-0.002 -0.602 2.961
-0.023 1.062 2.799
-0.160 3.008 0.167
-1.125 2.332 1.206
1.223 -1.154 2.477


In [7]:
# generate robot configurations

# import trajectory
traj_file="/home/armin/catkin_ws/src/kident2/src/traj.csv"
try:
    df = pd.read_csv(traj_file)
except Exception as e:
    print(e)
traj = df.to_numpy()  # shape: (num_joints, num_traj_points)
traj = traj[:, 1:]  # delete header
traj = traj.T
#print(traj)

In [8]:
# import robot values
theta_all = RobotDescription.dhparams["theta_nom"]
d_all = RobotDescription.dhparams["d_nom"]
r_all = RobotDescription.dhparams["r_nom"]
alpha_all = RobotDescription.dhparams["alpha_nom"]


In [22]:
def add_error_to_pose_measurement(pose_matrix, sigma_t, sigma_r):
    rvec, tvec = utils.mat2rvectvec(pose_matrix)
    
    r = R.from_rotvec(rvec)
    euler = r.as_euler('XYZ')
    euler_n = np.array(euler) + np.random.normal(0,sigma_r,3)
    r = R.from_euler('XYZ', euler_n)
    rvec_n = r.as_rotvec()

    tvec_n = np.array(tvec) + np.random.normal(0,sigma_t,3)
    return utils.H_rvec_tvec(rvec_n,tvec_n)


def get_model_error_local(obs1, obs2, theta, d, r, alpha):
    """
    calculate the difference between calculated and measured pose difference of two observtations
    """

    q1 = np.hstack((np.array(obs1["q"]), np.zeros(RobotDescription.dhparams['num_cam_extrinsic'])))
    q2 = np.hstack((np.array(obs2["q"]), np.zeros(RobotDescription.dhparams['num_cam_extrinsic'])))

    T_CM_1 = obs1['mat']
    T_CM_2 = obs2['mat']


    T_0_cam_1 = RobotDescription.get_T_0_cam(q1, theta, d, r, alpha)
    T_0_cam_2 = RobotDescription.get_T_0_cam(q2, theta, d, r, alpha)

    # perform necessary inversions
    T_MC_2 = np.linalg.inv(T_CM_2)
    T_cam_0_1 = np.linalg.inv(T_0_cam_1)

    # T_WM_1 = pe.T_W0 @ T_08_1 @ T_CM_1
    # if markerid == 2 or True:
    #     list_marker_locations.append([T_WM_1[0, 3], T_WM_1[1, 3], T_WM_1[2, 3]])

    D_meas = T_CM_1 @ T_MC_2
    D_nom = T_cam_0_1 @ T_0_cam_2
    delta_D = D_meas @ np.linalg.inv(D_nom)
    delta_D_skew = 0.5 * (delta_D - delta_D.T)

    drvec = np.array([delta_D_skew[2, 1], delta_D_skew[0, 2], delta_D_skew[1, 0]])
    dtvec = delta_D[0:3, 3]
    pose_error = np.concatenate((dtvec, drvec))

    return pose_error

In [51]:
def create_erroneous_simulated_obs(error_factor_r, error_factor_t):

    list_obs = []
    list_cam_positions = []

    # print out robot description
    #for item in RobotDescription.dhparams.keys():
    #    print(f"{item} : {RobotDescription.dhparams[item]}")
    last_frame = RobotDescription.dhparams['num_cam_extrinsic'] + RobotDescription.dhparams['num_joints']
    #print(f"num frames total: {last_frame}")

    # modify trajecory loaded from file
    traj_long = traj[0:30]

    # generate pseudo measurement data
    for q in traj_long:

        # choose 4 random markers for this configuration
        markers = random.choices(ids, k=4)

        joint_tfs = RobotDescription.get_joint_tfs(q)


        tm = TransformManager()

        for tf in joint_tfs:
            from_frame, to_frame, A2B = tf['from_frame'], tf['to_frame'], tf['mat']
            tm.add_transform(from_frame, to_frame, A2B)


        list_cam_positions.append(np.array(tm.get_transform(str(last_frame), 'world'))[0:3,3])

        for marker in markers:

            T_WM = dict_T_WM[marker]
            tm.add_transform("marker", "world", T_WM)

            T_CM = tm.get_transform('marker', str(last_frame))

            # add the error
            T_CM = add_error_to_pose_measurement(T_CM, error_factor_t, error_factor_r)

            obs = {"marker_id": marker,
                   "mat": T_CM,
                   "t": 0,
                   "q": q,
                   "interp_dist": 0}
            list_obs.append(obs)


    df = pd.DataFrame(list_obs)

    observations_file_str = 'observation_files/meas_err/observations_simulated_' + 'errors__r_' + f"{error_factor_r:.5f}" + '__t_' + f"{error_factor_t:.5f}"  + '.p'

    pd.to_pickle(df, observations_file_str)

    return observations_file_str

In [52]:
counter = 30

error_factor_t = 0
for error_factor_r in np.arange(0, 0.001, 0.0001):
    filestring = create_erroneous_simulated_obs(error_factor_r, error_factor_t)
    print(f"{counter}: r'"+filestring+"',")
    counter = counter + 1

counter = 60
error_factor_r = 0
for error_factor_t in np.arange(0, 0.001, 0.0001):
    filestring = create_erroneous_simulated_obs(error_factor_r, error_factor_t)
    print(f"{counter}: r'"+filestring+"',")
    counter = counter + 1

[array([-0.27427112,  0.27873783,  1.32040159]), array([0.30286482, 0.18928768, 1.41459308]), array([0.17346805, 0.34627149, 1.38617272]), array([-0.26000668,  0.08999532,  1.451236  ]), array([0.50635595, 0.42244621, 1.13543783]), array([0.93952838, 0.21972929, 0.82035291]), array([0.33257961, 0.13639237, 1.33621532]), array([-0.83629662, -0.14848208,  0.9031479 ]), array([0.83586585, 0.47714454, 0.68601857]), array([ 0.79481021, -0.06570084,  1.09860916]), array([-0.03316955, -0.35828598,  1.33780717]), array([-0.34997485, -0.49951948,  1.28133585]), array([-0.63513543,  0.16745582,  1.164253  ]), array([-0.50410969, -0.59083803,  0.89735579]), array([-0.19103579, -0.1858975 ,  1.35460889]), array([ 0.11902368, -0.02911608,  1.47728428]), array([-0.02216324, -0.13214435,  1.46700566]), array([0.34870203, 0.28803912, 1.34431587]), array([ 0.89314378, -0.02628362,  0.88604461]), array([-0.35160654,  0.23972972,  1.38963134]), array([0.38809794, 0.17442375, 1.32024919]), array([-0.13156

[array([-0.27427112,  0.27873783,  1.32040159]), array([0.30286482, 0.18928768, 1.41459308]), array([0.17346805, 0.34627149, 1.38617272]), array([-0.26000668,  0.08999532,  1.451236  ]), array([0.50635595, 0.42244621, 1.13543783]), array([0.93952838, 0.21972929, 0.82035291]), array([0.33257961, 0.13639237, 1.33621532]), array([-0.83629662, -0.14848208,  0.9031479 ]), array([0.83586585, 0.47714454, 0.68601857]), array([ 0.79481021, -0.06570084,  1.09860916]), array([-0.03316955, -0.35828598,  1.33780717]), array([-0.34997485, -0.49951948,  1.28133585]), array([-0.63513543,  0.16745582,  1.164253  ]), array([-0.50410969, -0.59083803,  0.89735579]), array([-0.19103579, -0.1858975 ,  1.35460889]), array([ 0.11902368, -0.02911608,  1.47728428]), array([-0.02216324, -0.13214435,  1.46700566]), array([0.34870203, 0.28803912, 1.34431587]), array([ 0.89314378, -0.02628362,  0.88604461]), array([-0.35160654,  0.23972972,  1.38963134]), array([0.38809794, 0.17442375, 1.32024919]), array([-0.13156

[array([-0.27427112,  0.27873783,  1.32040159]), array([0.30286482, 0.18928768, 1.41459308]), array([0.17346805, 0.34627149, 1.38617272]), array([-0.26000668,  0.08999532,  1.451236  ]), array([0.50635595, 0.42244621, 1.13543783]), array([0.93952838, 0.21972929, 0.82035291]), array([0.33257961, 0.13639237, 1.33621532]), array([-0.83629662, -0.14848208,  0.9031479 ]), array([0.83586585, 0.47714454, 0.68601857]), array([ 0.79481021, -0.06570084,  1.09860916]), array([-0.03316955, -0.35828598,  1.33780717]), array([-0.34997485, -0.49951948,  1.28133585]), array([-0.63513543,  0.16745582,  1.164253  ]), array([-0.50410969, -0.59083803,  0.89735579]), array([-0.19103579, -0.1858975 ,  1.35460889]), array([ 0.11902368, -0.02911608,  1.47728428]), array([-0.02216324, -0.13214435,  1.46700566]), array([0.34870203, 0.28803912, 1.34431587]), array([ 0.89314378, -0.02628362,  0.88604461]), array([-0.35160654,  0.23972972,  1.38963134]), array([0.38809794, 0.17442375, 1.32024919]), array([-0.13156

[array([-0.27427112,  0.27873783,  1.32040159]), array([0.30286482, 0.18928768, 1.41459308]), array([0.17346805, 0.34627149, 1.38617272]), array([-0.26000668,  0.08999532,  1.451236  ]), array([0.50635595, 0.42244621, 1.13543783]), array([0.93952838, 0.21972929, 0.82035291]), array([0.33257961, 0.13639237, 1.33621532]), array([-0.83629662, -0.14848208,  0.9031479 ]), array([0.83586585, 0.47714454, 0.68601857]), array([ 0.79481021, -0.06570084,  1.09860916]), array([-0.03316955, -0.35828598,  1.33780717]), array([-0.34997485, -0.49951948,  1.28133585]), array([-0.63513543,  0.16745582,  1.164253  ]), array([-0.50410969, -0.59083803,  0.89735579]), array([-0.19103579, -0.1858975 ,  1.35460889]), array([ 0.11902368, -0.02911608,  1.47728428]), array([-0.02216324, -0.13214435,  1.46700566]), array([0.34870203, 0.28803912, 1.34431587]), array([ 0.89314378, -0.02628362,  0.88604461]), array([-0.35160654,  0.23972972,  1.38963134]), array([0.38809794, 0.17442375, 1.32024919]), array([-0.13156

In [44]:
30: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00000__t_0.00000.p',
31: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00010__t_0.00000.p',
32: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00020__t_0.00000.p',
33: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00030__t_0.00000.p',
34: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00040__t_0.00000.p',
35: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00050__t_0.00000.p',
36: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00060__t_0.00000.p',
37: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00070__t_0.00000.p',
38: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00080__t_0.00000.p',
39: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00090__t_0.00000.p',
60: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00000__t_0.00000.p',
61: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00000__t_0.00010.p',
62: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00000__t_0.00020.p',
63: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00000__t_0.00030.p',
64: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00000__t_0.00040.p',
65: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00000__t_0.00050.p',
66: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00000__t_0.00060.p',
67: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00000__t_0.00070.p',
68: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00000__t_0.00080.p',
69: r'observation_files/meas_err/meas_err/observations_simulated_errors__r_0.00000__t_0.00090.p',


SyntaxError: invalid syntax (445454705.py, line 1)